In [1]:
lines = sc.textFile('./data/uid_iid')

In [2]:
target, etime, subset = "12-19-0", "12-18-23", {}
# target, etime, subset = "12-18-0", "12-17-23", {}
# target, etime, subset = "12-17-0", "12-16-23", {}
# target, etime, subset = "12-16-0", "12-15-23", {}
# target, etime, subset = "12-15-0", "12-14-23", {}
# target, etime, subset = "12-14-0", "12-13-23", {}
# target, etime, subset = "12-13-0", "12-12-23", {}
# target, etime, subset = "12-12-0", "12-11-23", {}
# target, etime, subset = "12-11-0", "12-10-23", {}
# target, etime, subset = "12-10-0", "12-09-23", {}
# target, etime, subset = "12-09-0", "12-08-23", {}
# target, etime, subset = "12-08-0", "12-07-23", {}
# target, etime, subset = "12-07-0", "12-06-23", {}
# target, etime, subset = "12-06-0", "12-05-23", {}
# target, etime, subset = "12-05-0", "12-04-23", {}
# target, etime, subset = "12-04-0", "12-03-23", {}
# target, etime, subset = "12-03-0", "12-04-23", {}
# target, etime, subset = "12-02-0", "12-01-23", {}
# target, etime, subset = "12-01-0", "11-30-23", {}

In [3]:
import fileinput

In [4]:
for line in fileinput.input("./data/tianchi_fresh_comp_train_item.csv"):
    subset[line.strip().split(",")[2]] = True

In [5]:
def extract1(line):
    import time
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(ict):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        return (uid+"\t"+ict,items)
    else:
        return ("","")

In [6]:
def extract2(items_list):
    import itertools
    items = []
    for i in items_list:
        items.extend(i)
    items = sorted(items, key=lambda x:x[0], reverse=True) if len(items)!=0 else []
    f, inf = [0]*26, 100
    buy = filter(lambda x:x[1]==4, items)
    last = buy[-1][0] if len(buy)!=0 else inf
    a1 = filter(lambda x:x[0]<last and x[1]==1, items)
    a2 = filter(lambda x:x[0]<last and x[1]==2, items)
    a3 = filter(lambda x:x[0]<last and x[1]==3, items)
    # 基本统计特征
    f[0] = a2[-1][0] if len(a2)!=0 else inf # 购买后最后一次加入收藏夹距离天数
    f[1] = a3[-1][0] if len(a3)!=0 else inf # 购买后最后一次加入购物车距离天数
    f[2] = len(a1) # 购买后点击次数
    f[3] = len(a2) # 购买后加收次数
    f[4] = len(a3) # 购买后加购次数
    f[5] = len(filter(lambda x:x[0]<=1, items)) # 最后1天交互次数
    f[6] = len(filter(lambda x:x[0]==2, items)) # 倒数第2天交互次数
    f[7] = len(filter(lambda x:x[0]==3, items)) # 倒数第3天交互次数
    f[8] = len(filter(lambda x:x[0]==4, items)) # 倒数第4天交互次数
    f[9] = len(filter(lambda x:x[0]<=7, items)) # 最后7天交互次数
    f[10] = len(buy) # 历史购买次数
    f[11] = last # 最后一次购买距离天数
    f[12] = len(set([item[0] for item in items if item[0]<=3])) # 最后3天内交互天数
    f[13] = len(set([item[0] for item in items if item[0]<=7])) # 最后1周内交互天数
    f[14] = len(set([item[0] for item in items if item[0]<=21])) # 最后3周内交互天数
    f[15] = items[-1][0] if len(items)!=0 else inf # 最后1次交互距离天数
    inter = [len(list(i)) for _,i in itertools.groupby(items, lambda x: x[0])]
    f[16] = len(inter) #交互天数
    f[17] = max(inter) if len(inter)!=0 else 0 #交互最多的一天交互次数
    f[18] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # 最后1天购买次数
    f[19] = len(filter(lambda x:x[0]<=2 and x[1]==4, items)) # 最后2天购买次数
    f[20] = len(filter(lambda x:x[0]<=3 and x[1]==4, items)) # 最后3天购买次数
    f[21] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # 最后7天购买次数
    f[22] = len(filter(lambda x:x[1]==1, items)) # 总点击次数
    f[23] = len(filter(lambda x:x[1]==2, items)) # 总添加收藏夹次数
    f[24] = len(filter(lambda x:x[1]==3, items)) # 总加购次数
    f[25] = items[0][0]-items[-1][0]+1 if len(items)!=0 else 0 # 第一次交互到最后一次交互的持续时间
    return "\t".join([str(i) for i in f])

In [7]:
counts = lines.map(lambda x : extract1(x))\
                .filter(lambda x : x[0]!="")\
                .groupByKey()\
                .map(lambda x : x[0]+"\t"+extract2(x[1]))

In [8]:
output = counts.saveAsTextFile("./feature/"+target+"/user_ict/")

Traceback (most recent call last):
  File "/tmp/kernel-PySpark-3bcbbd70-5252-4a60-a229-b1b60207ef35/pyspark_runner.py", line 205, in <module>
    output.reset()
AttributeError: 'NoneType' object has no attribute 'reset'
